# Day 08 - o1

In [1]:
def main():
    # Read the input map
    with open("input.txt") as f:
        grid = [line.rstrip('\n') for line in f]
        
    height = len(grid)
    width = len(grid[0]) if height > 0 else 0

    # Dictionary: frequency -> list of (x,y) positions of antennas
    antennas = {}
    for y in range(height):
        for x in range(width):
            c = grid[y][x]
            if c not in ['.', ' ']:  # Antenna characters are not '.' or ' '
                antennas.setdefault(c, []).append((x, y))
    
    # Set to hold unique antinode locations
    antinodes = set()

    # For each frequency, consider all pairs of antennas
    # Each pair of antennas (A,B) creates two antinodes:
    # Antinode1 = 2*A - B = (2*Ax - Bx, 2*Ay - By)
    # Antinode2 = 2*B - A = (2*Bx - Ax, 2*By - Ay)
    #
    # "A", "a", and digits are all distinct frequencies, so only pairs with identical chars.
    for freq, positions in antennas.items():
        # For efficiency, if there's only one antenna of that frequency, skip
        if len(positions) < 2:
            continue
        
        # Consider all pairs
        # We only need distinct pairs (i<j)
        for i in range(len(positions)):
            Ax, Ay = positions[i]
            for j in range(i+1, len(positions)):
                Bx, By = positions[j]
                
                # Compute the two antinodes
                antinode1 = (2*Ax - Bx, 2*Ay - By)
                antinode2 = (2*Bx - Ax, 2*By - Ay)
                
                # Check if they are within the map bounds
                if 0 <= antinode1[0] < width and 0 <= antinode1[1] < height:
                    antinodes.add(antinode1)
                if 0 <= antinode2[0] < width and 0 <= antinode2[1] < height:
                    antinodes.add(antinode2)
    
    # The result is the number of unique antinodes within the map
    print(len(antinodes))

if __name__ == "__main__":
    main()


299


## Part 2

In [2]:
from math import gcd, floor, ceil

def main():
    # Read the input map
    with open("input.txt") as f:
        grid = [line.rstrip('\n') for line in f]
        
    height = len(grid)
    width = len(grid[0]) if height > 0 else 0

    # Dictionary: frequency -> list of (x,y) positions of antennas
    antennas = {}
    for y in range(height):
        for x in range(width):
            c = grid[y][x]
            if c not in ['.', ' ']:  # antenna characters are not '.' or ' '
                antennas.setdefault(c, []).append((x, y))

    # We'll find all antinodes:
    # After the update:
    # An antinode occurs at any grid position "exactly in line with at least two antennas of the same frequency".
    # "In line" means that the point lies on the infinite line passing through at least two antennas of that frequency.
    #
    # Approach:
    # For each frequency, consider all pairs of antennas:
    # Each pair defines a line. We then find all lattice points (within the map) on that line.
    # We add these points to the set of antinodes.
    #
    # To avoid counting the same line multiple times (if multiple pairs of antennas define the same line),
    # we will store each line in a dictionary keyed by its canonical representation.
    #
    # Canonical line representation:
    # Given two distinct points (Ax,Ay) and (Bx,By):
    # dx = Bx - Ax, dy = By - Ay
    # g = gcd(dx, dy), dx //= g, dy //= g
    # Ensure a canonical direction: 
    #   If dx<0, multiply (dx,dy) by -1.
    #   If dx==0 and dy<0, multiply (dx,dy) by -1.
    # Then the line shift = dy*Ax - dx*Ay (this remains constant for any point on the same line)
    #
    # We'll store lines in a dict: line_map[(dx,dy,shift)] = True once encountered.
    #
    # After gathering all lines for a frequency, we find all points on each line within the grid.
    # To find points on the line:
    #   The line passes through (Ax,Ay) and is parameterized as:
    #       X(t) = Ax + t*dx
    #       Y(t) = Ay + t*dy
    #   for all integers t.
    # We must find all t such that 0 <= X(t) < width and 0 <= Y(t) < height.
    #
    # If dx==0, then line is vertical: X(t)=Ax is constant. 
    #   Check if 0<=Ax<width. If not, no points inside grid. If yes, then Y changes:
    #   Y(t) = Ay + t*dy. We find t range from vertical constraints.
    #
    # Similarly for horizontal lines if dy==0.
    #
    # Then we iterate over all integer t in that range, add points to antinodes.
    #
    # We'll do this for each frequency and combine results.
    #
    # Complexity: If there are many antennas of the same frequency, this could be large, but let's trust the input.

    antinodes = set()

    for freq, positions in antennas.items():
        if len(positions) < 2:
            # Only one antenna of that frequency means no line with another antenna,
            # but that single antenna itself is NOT an antinode because we need at least two antennas in line.
            # (The problem states that an antinode occurs at any grid position in line with at least TWO antennas.)
            continue
        
        line_map = {}
        # We'll pick pairs of antennas to define lines
        # After we find a pair, store the line in line_map
        # We'll also store one "anchor point" from the pair to use later for iteration.
        line_details = {}
        
        for i in range(len(positions)):
            Ax, Ay = positions[i]
            for j in range(i+1, len(positions)):
                Bx, By = positions[j]
                dx = Bx - Ax
                dy = By - Ay
                g = gcd(dx, dy)
                dx //= g
                dy //= g
                # Ensure canonical direction
                if dx < 0:
                    dx = -dx
                    dy = -dy
                elif dx == 0 and dy < 0:
                    dy = -dy
                shift = dy*Ax - dx*Ay
                line_key = (dx, dy, shift)
                if line_key not in line_map:
                    line_map[line_key] = (Ax, Ay, dx, dy)
        
        # Now iterate over each line and find all points on it within the grid
        for (dx, dy, shift), (Ax, Ay, lx, ly) in line_map.items():
            # (lx, ly) = (dx, dy) is direction, (Ax, Ay) is a point on the line
            # Parameterization: X = Ax + t*lx, Y = Ay + t*ly
            # Find t range for which X,Y in grid.
            
            # If lx == 0, line is vertical:
            if lx == 0:
                # X = Ax constant
                if 0 <= Ax < width:
                    # Y = Ay + t*ly
                    # 0 <= Ay + t*ly < height
                    # Solve inequalities for t:
                    # If ly > 0:
                    #   t >= -Ay/ly and t < (height - Ay)/ly
                    # If ly < 0 similarly
                    t_min_list = []
                    t_max_list = []
                    
                    if ly > 0:
                        t_min_list.append(ceil((0 - Ay)/ly))
                        t_max_list.append(floor((height-1 - Ay)/ly))
                    elif ly < 0:
                        t_min_list.append(ceil((height-1 - Ay)/ly))  # because dividing by negative flips inequality
                        t_max_list.append(floor((0 - Ay)/ly))
                    else:
                        # If ly == 0, the line is just a single point (Ax,Ay)
                        # Check if inside grid:
                        if 0 <= Ay < height:
                            antinodes.add((Ax, Ay))
                        continue
                    
                    t_min = max(t_min_list) if t_min_list else -10**9
                    t_max = min(t_max_list) if t_max_list else 10**9
                    
                    if t_min <= t_max:
                        for t in range(t_min, t_max+1):
                            Xp = Ax
                            Yp = Ay + t*ly
                            if 0 <= Yp < height:
                                antinodes.add((Xp, Yp))
            
            # If ly == 0, line is horizontal:
            elif ly == 0:
                # Y = Ay constant
                if 0 <= Ay < height:
                    # X = Ax + t*lx
                    # 0 <= Ax + t*lx < width
                    t_min_list = []
                    t_max_list = []
                    
                    if lx > 0:
                        t_min_list.append(ceil((0 - Ax)/lx))
                        t_max_list.append(floor((width-1 - Ax)/lx))
                    elif lx < 0:
                        t_min_list.append(ceil((width-1 - Ax)/lx))
                        t_max_list.append(floor((0 - Ax)/lx))
                    else:
                        # This would mean both lx=0 and ly=0 which can't happen with two distinct antennas
                        continue
                    
                    t_min = max(t_min_list) if t_min_list else -10**9
                    t_max = min(t_max_list) if t_max_list else 10**9
                    
                    if t_min <= t_max:
                        for t in range(t_min, t_max+1):
                            Xp = Ax + t*lx
                            Yp = Ay
                            if 0 <= Xp < width:
                                antinodes.add((Xp, Yp))
            
            else:
                # General case: lx !=0 and ly !=0
                # X in range: 0 <= Ax + t*lx < width
                # Y in range: 0 <= Ay + t*ly < height

                # For X:
                if lx > 0:
                    t_min_x = ceil((0 - Ax)/lx) if lx != 0 else -10**9
                    t_max_x = floor((width-1 - Ax)/lx) if lx != 0 else 10**9
                else:
                    t_min_x = ceil((width-1 - Ax)/lx)
                    t_max_x = floor((0 - Ax)/lx)

                # For Y:
                if ly > 0:
                    t_min_y = ceil((0 - Ay)/ly) if ly != 0 else -10**9
                    t_max_y = floor((height-1 - Ay)/ly) if ly != 0 else 10**9
                else:
                    t_min_y = ceil((height-1 - Ay)/ly)
                    t_max_y = floor((0 - Ay)/ly)

                t_min = max(t_min_x, t_min_y)
                t_max = min(t_max_x, t_max_y)

                if t_min <= t_max:
                    for t in range(t_min, t_max+1):
                        Xp = Ax + t*lx
                        Yp = Ay + t*ly
                        if 0 <= Xp < width and 0 <= Yp < height:
                            antinodes.add((Xp, Yp))

    # The result is the number of unique antinodes within the map
    print(len(antinodes))

if __name__ == "__main__":
    main()


1032
